In [25]:
import pandas as pd
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import re
import time
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [2]:

# URLS
shein_home_page_url="https://eur.shein.com/"

shein_clothing_url_format="https://eur.shein.com/new/New-In-Women-Clothing-sc-00200200.html?ici=eur_tab01navbar01menu06&scici=navbar_WomenHomePage~~tab01navbar01menu06~~1_6~~itemPicking_00200200~~~~0~~50001&srctype=category&userpath=category%3ENEW%20IN%3ENEW%20IN%20CLOTHING&page={0}"

shein_clothing_url="https://eur.shein.com/new/New-In-Women-Clothing-sc-00200200.html?ici=eur_tab01navbar01menu06&scici=navbar_WomenHomePage~~tab01navbar01menu06~~1_6~~itemPicking_00200200~~~~0~~50001&srctype=category&userpath=category%3ENEW%20IN%3ENEW%20IN%20CLOTHING"
#CONSTS

columns_list=['SKU',
                'Item Name',
                'Price',
                'Points Earnning',
                'Style',
                'Color',
                'Pattern Type',
                'Length',
                'Type',
                'Neckline',
                'Sleeve Length',
                'Sleeve Type',
                'Placket Type',
                'Details',
                'Season',
                'Composition',
                'Material',
                'Fabric',
                'Sheer',
                'Fit Type',
                'Closure Type',
                'Waist Line',
                'Panty Type',
                'Body',
                'Chest pad',
                'Bra Type',
                'Bottom Type']


In [3]:
# bs4 is not supporting dinamic data loading, so I had to use selenium web driver 
# to get the linked javascript to the html's.   
driver = webdriver.Chrome(executable_path=r"C:\Users\ofirr\Documents\שנה ב'\מבוא למדעי הנתונים\Fashion project\crawling\chromedriver.exe")

In [4]:
#the function returns the number of pages in the clothings section. 
def pages_amount():
    driver.get(shein_clothing_url)
    html=driver.page_source
    soup=BeautifulSoup(html)
    info=soup.find("span", {"class": "S-pagination__total"})
    pages=info.text
    pattern=r"Total (\d+) Pages"
    return (int(re.findall(pattern,pages)[0]))


In [5]:
# the function returns all of the items links from single page (1-40 pages)
def get_items_links(shein_page_url):
    links=[]
    colord_item_format="https://eur.shein.com/{0}-p-{1}-cat-1773.html"
    single_item_format="https://eur.shein.com{0}"
    driver.get(shein_page_url)
    html=driver.page_source
    soup=BeautifulSoup(html)
    items_info=soup.findAll("div", {"class": "S-product-item__info"})
    for item_info in items_info:
        color_container=item_info.find("section",{"class": "S-product-item__relatecolor-container"})
        if(color_container is not None):
            for lable in color_container.findAll("label"):
                aria=(lable.get("aria-label"))
                value=(lable.input.get("value"))
                aria=aria.replace(" ","-")
                str1=colord_item_format.format(aria,value)
                links.append(str1)
                
        else: 
            str2=single_item_format.format(item_info.a.get("href"))
            links.append(str2)
    return links

In [6]:
def get_url_for_each_page(pages_count):
    all_products_links=[]
    for i in range(1,pages_count):
        curr_url=shein_clothing_url_format.format(i)
        all_products_links.extend(get_items_links(curr_url)) #appending all lists together from each page
    return all_products_links

In [7]:
def get_details_from_single_item(item_link):
    description_dictionary={}
    driver.get(item_link)
    html=driver.page_source
    soup=BeautifulSoup(html)
    
    #SKU
    SKU=soup.find("div",{"class": "product-intro__head-sku"})
    SKU_text=SKU.text
    pattern=r"SKU: (\w+)"
    SKU_value=re.findall(pattern,SKU_text)[0]
    
    #Item Name
    
    Item_Name=soup.find("div",{"class": "product-intro__head-name"})
    Item_Name_text=Item_Name.text
    pattern=r"\n\s+(.*)\n"
    Item_Name_value=re.findall(pattern,Item_Name_text)[0]
    
    #Price
    price=soup.find("div",{"class": "from"})
    price_text=price.span.text
    price_text=price_text.replace(',','.')
    price_value=float(price_text[0:-1])
    

    #Points Earnning
    points_earnning=soup.find("span",{"class": "color-orange-tips"})
    if(points_earnning is not None):
        points_earnning_text=points_earnning.text
        pattern=r"(\w+)"
        points_earnning_string=re.findall(pattern,points_earnning_text)[0]
        points_earnning_value=int(points_earnning_string)
    else:
        points_earnning_value=0
        
    #Description 
    for i in soup.findAll("div",{"class":"product-intro__description-table-item"}):
        current_key=i.find("div",{"class":"key"}).text[0:-1]
        current_val=i.find("div",{"class":"val"}).text
        description_dictionary[current_key]=current_val
        
  
    
    dictionary={'SKU':SKU_value,
                'Item Name':Item_Name_value,
                'Price':price_value,
               }
    dictionary.update(description_dictionary)
    return(dictionary)

In [8]:
def insert_details_to_dataframe(dataframe,details):
    print(details)
    dataframe=dataframe.append( pd.Series(details), ignore_index=True)
    return( dataframe)

In [9]:

def build_dataframe_rows(dataframe,all_products_links):
    i=0
    for link in all_products_links:
        i=i+1
        item_dictionary=get_details_from_single_item(link)
        dataframe=insert_details_to_dataframe(dataframe,item_dictionary)
        if((i > 99) and ((i % 100) == 0)):
            dataframe.to_csv("shein_{0}.csv".format(i))
    return (dataframe)

In [11]:
import time

def main():

    df=pd.DataFrame(columns = columns_list)
#     pages_amount=pages_amount()
    all_products_links=get_url_for_each_page(2)
    df=build_dataframe_rows(df,all_products_links)
    df.to_csv("Shein_Wow.csv")
    
main()

In [14]:
df=pd.read_csv("Shein_Wow.csv",index_col=0)
copy=df.drop_duplicates(keep='first',ignore_index=False).copy()
def fill_empty_values(df):
    df.fillna("Empty", inplace=True)
fill_empty_values(copy)


In [15]:
# Because there are few types of details in each item I want to separate the types to diffrent columns. 
#There are many diffrent details types so I took only over 100 freaqwency types.

def divide_details_col():
    dictionaroro= {}
    sorted_dict={}
    list1=[]
    for i in copy['Details']:
        for j in i.split(', '):
            if(j not in dictionaroro.keys()):
                dictionaroro[j]=1 
            else:
                dictionaroro[j]=dictionaroro[j]+1
    sorted_keys = sorted(dictionaroro, key=dictionaroro.get,reverse=True)

    for w in sorted_keys:
        if(dictionaroro[w]>=100 and w!='Empty'):
            sorted_dict[w] = dictionaroro[w]
            list1.append(w)
            
    for x in list1:
        copy[x]=(copy['Details'].str.contains(x)).astype(int)

divide_details_col()

In [20]:
def remove_empty_cols(col_list):   
    copy.drop(axis=1, index=None, columns=col_list, inplace=True)

col_list=['Details','Composition','Bra Type','Chest pad','Panty Type','Arabian Clothing','Care Instructions']
remove_empty_cols(col_list)

In [23]:
#remove colors with low frequency.
val = copy['Color'].value_counts()
y = val[val < 30].index
copy['Color'] =  copy['Color'].replace({x:'other' for x in y})
copy['Color'].nunique()

#remove style type with low frequency.
val = copy['Style'].value_counts()
y = val[val < 30].index
copy['Style'] =  copy['Style'].replace({x:'other' for x in y})
copy['Style'].nunique()


3

In [26]:
def find_string_cols(df):
    list_cols=[]
    for col in df.columns: 
        if((df[col].dtype=='object') and (col not in ['SKU','Item Name','Details'])):
            list_cols.append(col)
    return list_cols
def encode_string_cols(df,list_cols):
    le = preprocessing.LabelEncoder()

    for col in list_cols:
        df[col].fillna("Empty", inplace=True)
        df[col] = le.fit_transform(df[col])
    return df,le
copy,le=encode_string_cols(copy,find_string_cols(copy))

In [27]:
copy.to_csv( "Shein_Wow_Data.csv", index=False, encoding='utf-8-sig')